In [1]:
import sys
print (sys.executable)

C:\Users\GADILP50\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://listado.mercadolibre.com.pe/celulares#D[A:celulares]'

encabezados = {
"user-agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36" # Para que no sea un robot y no identifique que es web scraping
}
response = requests.get(url,headers=encabezados)
content = response.content

In [4]:
soup = BeautifulSoup(content, 'html.parser')
#print(soup.prettify())
prod = soup.find_all('h2', class_='ui-search-item__title shops__item-title')
productos = list()

for i in prod:
   productos.append(i.text)

In [5]:
prec= soup.find_all('span', class_='price-tag-fraction')
precios = list()

for i in prec:
   precios.append(i.text)

In [6]:
prec_ini= soup.find_all('span', class_='price-tag-text-sr-only')
prec_inicial = list()

for i in prec_ini:
    prec_inicial.append(i.text)

## Crear listas vacías para almacenar los datos

In [7]:
precios_iniciales = []
precios_finales = []

## Iterar sobre la lista de precios

In [8]:
i = 0
while i < len(prec_inicial):
    precio_actual = prec_inicial[i]
    if precio_actual.startswith('Antes'):
        precio_inicial = precio_actual.split('Antes: ')[1]
        precio_final = prec_inicial[i + 1] if i + 1 < len(prec_inicial) else ''
        precios_iniciales.append(precio_inicial)
        precios_finales.append(precio_final)
        i += 2 # Saltar al siguiente par de elementos
    else:
        # No hay precio inicial, asignar como 'Sin descuento'
        precios_iniciales.append('Sin descuento')
        precios_finales.append(precio_actual)
        i += 1 # Saltar al siguiente elemento

df_total = pd.DataFrame({'Nombre Producto':productos,'Precio inicial':precios_iniciales,
'Precio final': precios_finales})

In [9]:
import re

## Extraer valores numéricos y convertir a decimales

In [10]:
df_total['Precio inicial'] = df_total['Precio inicial'].apply(lambda x: re.findall(r'\d+.\d+|\d+', x)[0] if re.findall(r'\d+.\d+|\d+', x) else 0)
df_total['Precio final'] = df_total['Precio final'].apply(lambda x: re.findall(r'\d+.\d+|\d+', x)[0] if re.findall(r'\d+.\d+|\d+', x) else 0)

## Convertir columnas a tipo numérico

In [11]:

df_total['Precio inicial'] = pd.to_numeric(df_total['Precio inicial'])
df_total['Precio final'] = pd.to_numeric(df_total['Precio final'])

## Ordenar DataFrame por Precio Final en orden ascendente

In [12]:
df_total = df_total.sort_values('Precio final', ascending=True)

In [13]:
print(df_total)

Empty DataFrame
Columns: [Nombre Producto, Precio inicial, Precio final]
Index: []
